In [20]:
# %load CNN-retinex.py
"""
Created on Thu Sep 28 14:44:30 2017

@author: YUCHIH
"""
import numpy as np
import h5py as h5py
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import load_model
import matplotlib.pyplot as plt
import pandas as pd

#Initialize the CNN
classifier = Sequential()

#step1
classifier.add(Convolution2D(16,3,3,input_shape= (64,64,3), activation = 'relu'))

#step2
classifier.add(MaxPooling2D(pool_size = (2,2)))

#step1
classifier.add(Convolution2D(32,3,3, activation = 'relu'))

#step2
classifier.add(MaxPooling2D(pool_size = (2,2)))

#step1
classifier.add(Convolution2D(64,3,3, activation = 'relu'))

#step2
classifier.add(MaxPooling2D(pool_size = (2,2)))

#step1
classifier.add(Convolution2D(128,3,3, activation = 'relu'))

#step2
classifier.add(MaxPooling2D(pool_size = (2,2)))
#step3
classifier.add(Flatten())


#step4
classifier.add(Dense(output_dim = 100, activation = 'relu'))
classifier.add(Dropout(p=0.03))
classifier.add(Dense(output_dim = 1, activation = 'sigmoid'))

#compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# import OS
import os 

#setwd
os.chdir('C:/Users/xd944/Documents/20171212/retinex/CNN-result/')

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(horizontal_flip=True)

training_set = train_datagen.flow_from_directory(
        'C:/Users/xd944/Documents/20171212/retinex/training/',
        target_size=(64,64),
        batch_size=12,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'C:/Users/xd944/Documents/20171212/retinex/testing/',
        target_size=(64,64),
        batch_size=32,
        class_mode='binary')
Specificity =np.array([None]*30,dtype = float)
Sensitivity =np.array([None]*30,dtype = float)
Accuracy =np.array([None]*30,dtype = float)
a = 0
for a in range(0,30):
    history = classifier.fit_generator(training_set,
            steps_per_epoch=24,
            epochs=20,
            validation_data=test_set,
            validation_steps=6)
    
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig("CNN_Accuracy_retinex_"+str(a)+".png")
    
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig("CNN_Loss_retinex_"+str(a)+".png")
    
    classifier.save('20171212_CNN_retinex_'+str(a)+'.h5')
    
    
    training_set.class_indices
    datapath = 'C:/Users/xd944/Documents/20171212/retinex/Predict/'
    #classifier = load_model('20171212_CNN_retinex.h5')
    
    
    from keras.preprocessing import image
    img_list = os.listdir(datapath)
        
    results = []
    for i in range(0,len(img_list),1):
        test_image = image.load_img(datapath + img_list[i],target_size=(64,64))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image,axis = 0)
        results.append(classifier.predict(test_image))
        if results[i][0] == 0:
            prediction = 'Glaucoma'
        else:
            prediciton = 'Healthy'
        
    Final = np.hstack(results)
    Final = Final.T
    
    name_list = img_list
    i=0
    for i in range(0,len(img_list)):
        name_list[i] = img_list[i][0:9] #+ new[i][-8:]
        i=i+1
        
    Final_list = np.column_stack([name_list,Final])
    index = ['Row'+str(i) for i in range(1, len(Final_list)+1)]
    test = pd.DataFrame(data = Final_list[0:,0:],columns = ["ID","Prediction"],index=index)
    test = test.astype(dtype= {"ID":"object","Prediction":"float"})
    Final_list = pd.DataFrame(data = test.groupby(["ID"], as_index=False).mean())
    
    list_AB=[None]*len(Final_list)
    i=0
    for A in Final_list.iloc[0:,0]:
        list_AB[i] = A[0]
        i=i+1
            
    count_A = list_AB.count("A")
    count_B = list_AB.count("B")
    
    Final_list[Final_list["Prediction"] >= 0.5] = 1
    Final_list[Final_list["Prediction"] < 0.5] = 0
    
    H = count_A
    G = count_B
    Specificity[a] = sum(Final_list.iloc[0:H,1])/H
    Sensitivity[a] = 1 - sum(Final_list.iloc[H:len(Final_list),1])/G
    Accuracy[a] = (Specificity[a]*H + Sensitivity[a]*G)/(H+G)
    print("Specificity: {}".format(Specificity[a]))
    print("Sensitivity: {}".format(Sensitivity[a]))
    print("Accuracy: {}".format(Accuracy[a]))
    
f = open("output_"+'.txt',"w") #opens file with name of "test.txt"
f.write("Sensitivity:"+ str(np.mean(Sensitivity)) +"\n")
f.write("Specificity:"+ str(np.mean(Specificity)) + "\n")
f.write("Accuracy:"+  str(np.mean(Accuracy)))
f.close()
    


C:\Users\xd944\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), input_shape=(64, 64, 3..., activation="relu")`
C:\Users\xd944\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
C:\Users\xd944\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
C:\Users\xd944\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
C:\Users\xd944\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:50: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=100, activation="relu")`
C:\Users\xd944\Anaconda3\envs\tensorflow\lib\site-packa

Found 482 images belonging to 2 classes.
Found 196 images belonging to 2 classes.
Epoch 1/20
24/24 [==============================] - 6s 265ms/step - loss: 0.4081 - acc: 0.8783 - val_loss: 1.3432 - val_acc: 0.9167
Epoch 2/20
24/24 [==============================] - 5s 229ms/step - loss: 0.3088 - acc: 0.9132 - val_loss: 1.3432 - val_acc: 0.9167
Epoch 3/20
24/24 [==============================] - 6s 241ms/step - loss: 0.2802 - acc: 0.9131 - val_loss: 1.3432 - val_acc: 0.9167
Epoch 4/20
24/24 [==============================] - 6s 256ms/step - loss: 0.2792 - acc: 0.9165 - val_loss: 1.3432 - val_acc: 0.9167
Epoch 5/20
24/24 [==============================] - 6s 239ms/step - loss: 0.2854 - acc: 0.9028 - val_loss: 1.3245 - val_acc: 0.9167
Epoch 6/20
24/24 [==============================] - 7s 311ms/step - loss: 0.2359 - acc: 0.9028 - val_loss: 1.0309 - val_acc: 0.9323
Epoch 7/20
24/24 [==============================] - 6s 234ms/step - loss: 0.1378 - acc: 0.9478 - val_loss: 0.2321 - val_acc: 0

dict_keys(['val_acc', 'val_loss', 'acc', 'loss'])
Specificity: 1.0
Sensitivity: 1.0
Accuracy: 1.0
Epoch 1/20
24/24 [==============================] - 7s 309ms/step - loss: 0.0045 - acc: 1.0000 - val_loss: 0.0839 - val_acc: 0.9948
Epoch 2/20
24/24 [==============================] - 5s 227ms/step - loss: 0.0064 - acc: 1.0000 - val_loss: 0.1670 - val_acc: 0.9896
Epoch 3/20
24/24 [==============================] - 6s 233ms/step - loss: 0.0019 - acc: 1.0000 - val_loss: 0.1694 - val_acc: 0.9896
Epoch 4/20
24/24 [==============================] - 6s 231ms/step - loss: 0.0367 - acc: 0.9896 - val_loss: 0.1670 - val_acc: 0.9896
Epoch 5/20
24/24 [==============================] - 5s 224ms/step - loss: 0.0816 - acc: 0.9687 - val_loss: 0.3358 - val_acc: 0.9792
Epoch 6/20
24/24 [==============================] - 7s 290ms/step - loss: 0.0599 - acc: 0.9722 - val_loss: 0.1670 - val_acc: 0.9896
Epoch 7/20
24/24 [==============================] - 5s 223ms/step - loss: 0.0230 - acc: 0.9931 - val_loss: 0.3

Epoch 20/20
24/24 [==============================] - 5s 223ms/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.3353 - val_acc: 0.9792
dict_keys(['val_acc', 'val_loss', 'acc', 'loss'])
Specificity: 1.0
Sensitivity: 1.0
Accuracy: 1.0
Epoch 1/20
24/24 [==============================] - 7s 301ms/step - loss: 3.3924e-04 - acc: 1.0000 - val_loss: 0.4161 - val_acc: 0.9740
Epoch 2/20
24/24 [==============================] - 5s 217ms/step - loss: 5.4528e-04 - acc: 1.0000 - val_loss: 0.2500 - val_acc: 0.9844
Epoch 3/20
24/24 [==============================] - 5s 215ms/step - loss: 1.6701e-04 - acc: 1.0000 - val_loss: 0.4161 - val_acc: 0.9740
Epoch 4/20
24/24 [==============================] - 5s 228ms/step - loss: 0.0025 - acc: 1.0000 - val_loss: 0.9134 - val_acc: 0.9427
Epoch 5/20
24/24 [==============================] - 5s 203ms/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.2500 - val_acc: 0.9844
Epoch 6/20
24/24 [==============================] - 6s 267ms/step - loss: 3.1183e-04 - acc: 1.000

Epoch 17/20
24/24 [==============================] - 5s 206ms/step - loss: 2.6742e-04 - acc: 1.0000 - val_loss: 0.6643 - val_acc: 0.9583
Epoch 18/20
24/24 [==============================] - 7s 280ms/step - loss: 5.1719e-04 - acc: 1.0000 - val_loss: 0.7473 - val_acc: 0.9531
Epoch 19/20
24/24 [==============================] - 5s 211ms/step - loss: 7.3201e-04 - acc: 1.0000 - val_loss: 0.4982 - val_acc: 0.9688
Epoch 20/20
24/24 [==============================] - 5s 215ms/step - loss: 1.5916e-04 - acc: 1.0000 - val_loss: 0.4982 - val_acc: 0.9688
dict_keys(['val_acc', 'val_loss', 'acc', 'loss'])
Specificity: 1.0
Sensitivity: 1.0
Accuracy: 1.0
Epoch 1/20
24/24 [==============================] - 7s 274ms/step - loss: 1.8409e-04 - acc: 1.0000 - val_loss: 0.5812 - val_acc: 0.9635
Epoch 2/20
24/24 [==============================] - 5s 204ms/step - loss: 6.0878e-04 - acc: 1.0000 - val_loss: 0.3321 - val_acc: 0.9792
Epoch 3/20
24/24 [==============================] - 5s 197ms/step - loss: 1.8766e-

24/24 [==============================] - 4s 183ms/step - loss: 2.2885e-04 - acc: 1.0000 - val_loss: 0.3321 - val_acc: 0.9792
Epoch 14/20
24/24 [==============================] - 5s 208ms/step - loss: 6.4439e-05 - acc: 1.0000 - val_loss: 0.4982 - val_acc: 0.9688
Epoch 15/20
24/24 [==============================] - 5s 188ms/step - loss: 1.8316e-05 - acc: 1.0000 - val_loss: 0.4982 - val_acc: 0.9688
Epoch 16/20
24/24 [==============================] - 5s 210ms/step - loss: 1.3250e-04 - acc: 1.0000 - val_loss: 0.4152 - val_acc: 0.9740
Epoch 17/20
24/24 [==============================] - 4s 187ms/step - loss: 7.0545e-05 - acc: 1.0000 - val_loss: 0.3321 - val_acc: 0.9792
Epoch 18/20
24/24 [==============================] - 6s 249ms/step - loss: 6.1860e-04 - acc: 1.0000 - val_loss: 0.9134 - val_acc: 0.9427
Epoch 19/20
24/24 [==============================] - 4s 186ms/step - loss: 2.1131e-04 - acc: 1.0000 - val_loss: 0.9964 - val_acc: 0.9375
Epoch 20/20
24/24 [==============================] - 

24/24 [==============================] - 5s 198ms/step - loss: 3.3863e-05 - acc: 1.0000 - val_loss: 0.1661 - val_acc: 0.9896
Epoch 10/20
24/24 [==============================] - 4s 180ms/step - loss: 4.9124e-05 - acc: 1.0000 - val_loss: 0.3321 - val_acc: 0.9792
Epoch 11/20
24/24 [==============================] - 5s 224ms/step - loss: 1.1753e-04 - acc: 1.0000 - val_loss: 0.3321 - val_acc: 0.9792
Epoch 12/20
24/24 [==============================] - 5s 199ms/step - loss: 8.4654e-05 - acc: 1.0000 - val_loss: 0.5812 - val_acc: 0.9635
Epoch 13/20
24/24 [==============================] - 4s 184ms/step - loss: 6.6318e-06 - acc: 1.0000 - val_loss: 0.4152 - val_acc: 0.9740
Epoch 14/20
24/24 [==============================] - 4s 184ms/step - loss: 6.4695e-05 - acc: 1.0000 - val_loss: 0.5812 - val_acc: 0.9635
Epoch 15/20
24/24 [==============================] - 4s 181ms/step - loss: 8.0953e-06 - acc: 1.0000 - val_loss: 0.9134 - val_acc: 0.9427
Epoch 16/20
24/24 [==============================] - 

Epoch 5/20
24/24 [==============================] - 5s 196ms/step - loss: 1.5112e-05 - acc: 1.0000 - val_loss: 0.4152 - val_acc: 0.9740
Epoch 6/20
24/24 [==============================] - 5s 226ms/step - loss: 3.2239e-05 - acc: 1.0000 - val_loss: 0.3321 - val_acc: 0.9792
Epoch 7/20
24/24 [==============================] - 5s 192ms/step - loss: 8.5908e-06 - acc: 1.0000 - val_loss: 0.3321 - val_acc: 0.9792
Epoch 8/20
24/24 [==============================] - 4s 180ms/step - loss: 9.2100e-06 - acc: 1.0000 - val_loss: 0.3321 - val_acc: 0.9792
Epoch 9/20
24/24 [==============================] - 5s 198ms/step - loss: 1.3735e-05 - acc: 1.0000 - val_loss: 0.3321 - val_acc: 0.9792
Epoch 10/20
24/24 [==============================] - 5s 196ms/step - loss: 0.0055 - acc: 0.9930 - val_loss: 0.1670 - val_acc: 0.9896
Epoch 11/20
24/24 [==============================] - 6s 249ms/step - loss: 0.0190 - acc: 0.9931 - val_loss: 0.2500 - val_acc: 0.9844
Epoch 12/20
24/24 [==============================] - 5

TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')

In [1]:
str(np.mean(Sensitivity))

NameError: name 'np' is not defined

In [22]:
np.mean(Specificity)

1.0

In [23]:
np.mean(Accuracy)

0.9951388888888888

In [26]:
f = open("output_"+'.txt',"w") #opens file with name of "test.txt"
f.write("Sensitivity:"+ str(np.mean(Sensitivity)) +"\n")
f.write("Specificity:"+ str(np.mean(Specificity)) + "\n")
f.write("Accuracy:"+  str(np.mean(Accuracy)))
f.close()